In [2]:
// Requiring all the Imports
import org.apache.spark.streaming.StreamingContext
import org.apache.spark.SparkContext
import org.apache.spark.streaming.Seconds
import org.apache.log4j.Level
import org.apache.spark.sql.SparkSession
// creating a Spark Session
val spark=SparkSession.builder().master("local[2]").appName("MyStreamingApplication")
.config("spark.streaming.stopGracefullyOnShutdown",true)
.config("spark.sql.shuffle.partitions",4)
.getOrCreate()

//read the data from socket
val linesDf=spark.readStream.format("socket").option("host","localhost").option("port","6767").load()


import org.apache.spark.sql.types._
val orderSchema=StructType(List(
StructField("order_id",IntegerType),
StructField("order_date",TimestampType),
StructField("order_customer_id",IntegerType),
StructField("order_status",StringType),
StructField("order_amount",IntegerType)))
import org.apache.spark.sql.functions._

// procesing of data
val valueDf=linesDf.select(from_json(col("value"),orderSchema).alias("value"))
valueDf.printSchema()
//val redefinedDf=valueDf.select("value")

val finalDf=valueDf.select("value.*")
val windowDf=finalDf.groupBy(window(col("order_date"),"15 minute")).agg(sum("order_amount").alias("Total Invoice"))
val outputDf=windowDf.select(window.start,window.end,Total Invoice)
val orders=outputDf.writeStream.format("console").outputMode("update")option("checkpointLocation","Location1").trigger(Trigger.processingTime("15 second")).start()



Compile Error: <console>:59: error: ambiguous reference to overloaded definition,
both method window in object functions of type (timeColumn: org.apache.spark.sql.Column, windowDuration: String)org.apache.spark.sql.Column
and  method window in object functions of type (timeColumn: org.apache.spark.sql.Column, windowDuration: String, slideDuration: String)org.apache.spark.sql.Column
match expected type ?
       val outputDf=windowDf.select(window.start,window.end,Total Invoice)
                                    ^
<console>:59: error: ambiguous reference to overloaded definition,
both method window in object functions of type (timeColumn: org.apache.spark.sql.Column, windowDuration: String)org.apache.spark.sql.Column
and  method window in object functions of type (timeColumn: org.apache.spark.sql.Column, windowDuration: String, slideDuration: String)org.apache.spark.sql.Column
match expected type ?
       val outputDf=windowDf.select(window.start,window.end,Total Invoice)
             